In [ ]:
!pip install accelerate
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.7/664.8 MB 7.1 MB/s eta 0:01:08

In [ ]:
# #Converting .mp4 to wav file
# from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

def convert_video_to_audio(video_path, audio_path):
    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        audio.write_audiofile(audio_path, codec='pcm_s16le')  # Saving as WAV     format
        print("Audio file successfully created!")
    except Exception as e:
        print(f"Error: {e}")


video_path = "ex1.mp4"  # TODO:Upgrade to support other codecs
file_name = video_path[9:-4]
audio_path = video_path[0:-4] + ".wav"
print(audio_path)
# convert_video_to_audio(video_path, audio_path)

ex1.wav


In [ ]:
#Speech To Text
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model_for_speech_2_text = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model_for_speech_2_text.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_for_speech_2_text,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe(audio_path)
Transcript = result['text']
print(result)
print(result.keys())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


FileNotFoundError: [Errno 2] No such file or directory: 'ex1.wav'

In [ ]:
Transcript

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_t = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer_t = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

Translated_Transcript = []

splited_transcript = Transcript.split(".")

for i in splited_transcript:
  tokenizer_t.src_lang = "en"
  encoded_hi = tokenizer_t(i, return_tensors="pt")
  generated_tokens = model_t.generate(**encoded_hi, forced_bos_token_id=tokenizer_t.get_lang_id("hi")).to(device)
  l = tokenizer_t.batch_decode(generated_tokens, skip_special_tokens=True)
  Translated_Transcript.append(l)

In [ ]:
Translated_Transcript

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install TTS

In [ ]:
single_string = ' '.join([sentence[0] for sentence in Translated_Transcript])

print(single_string)

In [ ]:
from TTS.api import TTS

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")
tts.to(device)


text_translated = single_string
curr_file_path = "/content/output1.wav"
tts.tts_to_file(text=text_translated,
              file_path=curr_file_path,
              speaker_wav = "/content/ex1.wav",
              language="hi")

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip

# Load the original video
original_video = VideoFileClip(video_path)

# Load the new audio file
new_audio = AudioFileClip("output1.wav")

# Replace the audio of the original video with the new audio
final_clip = original_video.set_audio(new_audio)

# Write the final video to a new file
final_clip.write_videofile("final_video.mp4")